## YOLOV5 FINE TUNING

### COCO DATASET

In [ ]:
import os

# Set paths
DATASET_YAML = "dataset.yaml"  # Update this if your dataset.yaml is in a different location
MODEL_PATH = "yolov5nu.pt"  # Path to your pre-trained YOLOv5 model
EPOCHS = 100  # Number of training epochs (adjust as needed)
BATCH_SIZE = 16  # Adjust based on your GPU memory
IMG_SIZE = 640  # Input image size (default: 640x640)
DEVICE = "cuda"  # Set to "cpu" if you don’t have a GPU

# Run YOLOv5 training
os.system(f"python yolov5/train.py --img {IMG_SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} "
          f"--data {DATASET_YAML} --weights {MODEL_PATH} --device {DEVICE} --workers 4")

print("Training completed!")


### POTHOLE DATASET

In [ ]:
import os

# Set paths
DATASET_YAML = "pothole.yaml"  # Make sure this points to your dataset.yaml
MODEL_PATH = "yolov5nu.pt"  # Your pre-trained YOLOv5 model
EPOCHS = 100  # Adjust epochs as needed
BATCH_SIZE = 16  # Modify based on your GPU memory
IMG_SIZE = 640  # Input image size (default: 640x640)
DEVICE = "cuda" 

# Run YOLOv5 training
os.system(f"python yolov5/train.py --img {IMG_SIZE} --batch {BATCH_SIZE} --epochs {EPOCHS} "
          f"--data {DATASET_YAML} --weights {MODEL_PATH} --device {DEVICE} --workers 4")

print("✅ Training completed!")


## EASY OCR 

### ICDAR 2015 DATASET